In [4]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import gc

In [59]:
os.listdir('flattened_chunks_ref/')

['references6.pickle',
 'references0.pickle',
 'references5.pickle',
 'references3.pickle',
 'references8.pickle',
 'references2.pickle',
 'references1.pickle',
 'references7.pickle',
 'references9.pickle',
 'references10.pickle',
 'references_merged.pickle',
 'references4.pickle']

In [60]:
# os.remove(MERGED_PICKLE_PATH)

In [61]:
MERGED_PICKLE_PATH = 'flattened_chunks_ref/references_merged.pickle'
try:
    dblp_flattened_ref_df = pd.read_pickle(MERGED_PICKLE_PATH)
except:
    chunks_df = []
    pickle_files = os.listdir('flattened_chunks_ref/')
    for file in pickle_files:
        print('flattened_chunks_ref/'+file)
        temp_df = pd.read_pickle('flattened_chunks_ref/'+file)
        chunks_df.append(temp_df)

    print(len(chunks_df))
    dblp_flattened_ref_df = pd.concat(chunks_df)
    dblp_flattened_ref_df.to_pickle(MERGED_PICKLE_PATH)


In [62]:
dblp_flattened_ref_df.head()

,id,references
0,089cb00e-3d11-4e9b-8370-d5ccbddba5dd,624e102b-e716-44f3-8957-866f8010c4d4
1,089cb00e-3d11-4e9b-8370-d5ccbddba5dd,6527f747-563e-4e5f-b9a1-44ea23adbf26
2,089cb00e-3d11-4e9b-8370-d5ccbddba5dd,f0b75351-be10-4961-a1b7-414945b42ab9
3,089cb5a9-f57d-4f99-ae15-7d7d8d557abf,17df7757-4086-48a6-bc72-626ac568ad09
4,089cb5a9-f57d-4f99-ae15-7d7d8d557abf,2bc197a7-eacb-4458-bafa-a04b77cf8790


In [63]:
dblp_flattened_ref_df.shape

(25529848, 2)

In [64]:
dblp_flattened_ref_df = dblp_flattened_ref_df[dblp_flattened_ref_df.references != "[]"]

In [65]:
dblp_flattened_ref_df.shape

(25166985, 2)

In [71]:
gc.collect()

69

In [78]:
graph = dict()
max_len = 0
for idx, row in dblp_flattened_ref_df.iterrows():
    if row['references'] not in graph:
        graph[row['references']] = []
    graph[row['references']].append(row['id'])

In [79]:
import pickle
pickle.dump(graph, open('graph.pickle', 'wb'))

In [84]:
len(graph['624e102b-e716-44f3-8957-866f8010c4d4'])
len(graph)

1985920

In [66]:
import networkx as nx

In [67]:
# g = nx.from_pandas_dataframe(dblp_flattened_ref_df, "references", "id",
#                          edge_attr=True, create_using=nx.DiGraph())

In [68]:
g

In [42]:
# len(list(nx.dfs_edges(g)))

897

In [48]:
# edges = list(nx.bfs_edges(g, '624e102b-e716-44f3-8957-866f8010c4d4'))

In [49]:
len(edges)

2284885

In [52]:
# edges1 = list(nx.bfs_edges(g, 'f0b75351-be10-4961-a1b7-414945b42ab9'))

In [ ]:
# h = nx.hits(g)

In [151]:
import math

In [152]:
def dp():
    dp.memo = {}

def reach(p, h, adj, visited, cutoff):
    if p in visited:
        return None
    
    if h > cutoff:
        return None

    visited.add(p)

    if (p, h) in dp.memo:
        return dp.memo[(p, h)]

    rp = len(adj.get(p,[])) * (math.sqrt(h))
    np = 1
    hp = h
    for q in adj.get(p,[]):
        reach_tuple = reach(q, h + 1, adj, visited, cutoff)
        if reach_tuple is None:
            continue

        rq, hq, nq = reach_tuple
        rp = rp + rq
        hp = max(hp , hq)   
        np = np + nq

    dp.memo[(p, h)] = (rp, hp, np)

    return (rp, hp, np)

In [160]:
gc.collect()

0

In [161]:
%time
dp()
reach(p='624e102b-e716-44f3-8957-866f8010c4d4', h=0, adj=graph, visited=set([]), cutoff=5)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


(1428013.1232224863, 5, 71596)

In [162]:
%time
dp()
reach(p='f0b75351-be10-4961-a1b7-414945b42ab9', h=0, adj=graph, visited=set([]), cutoff=5)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


(855146.6542541811, 5, 41180)

In [165]:
reach_vals = dict()
for p in graph.keys():
    r = reach(p=p, h=0, adj=graph, visited=set([]), cutoff=5)
    reach_vals[p] = r

{'1736fc7c-c936-4f2b-87ef-30dad4fdf886': (133484.2689338016, 5, 8815),
 '5647df54-107b-4a48-a818-8d4f9c0be37b': (5178.759164246329, 5, 751),
 'a3866faf-6b19-4f84-8f68-2ea2f0873199': (871.9630786536786, 5, 172),
 'b5c0acb1-cb42-4473-8e92-8f7c9ae334b0': (5.414213562373095, 3, 6),
 'dcf4b907-a78c-4076-a216-62f684b696fb': (693.1064995930915, 5, 107)}

In [1]:
import pickle
reach_vals = pickle.load(open('reach_vals_with_category_10.pickle', 'rb'))

In [2]:
len(reach_vals)

1985920

In [5]:
reach_df = pd.DataFrame(reach_vals)

In [6]:
reaches = []
for key, val in reach_vals.items():
    reaches.append((key, val[0], val[1], val[2]))

In [7]:
reaches[:10]

[('604de395-762d-4a73-b1f2-43a1c417834f', 0, 1, 2),
 ('332594be-9822-413c-aa98-004fda59e0bd', 0, 1, 2),
 ('6d27af8a-f73b-4c24-bb9e-0dd3f0d6e66d', 7, 3, 4),
 ('f478b5dd-1632-47ad-b57a-bee020bc5777', 4, 1, 2),
 ('575c4a39-d788-43ba-9b59-7e58557cde57', 0, 1, 2),
 ('8eb50f74-89a2-4683-9a9d-e7615c74d805', 0, 1, 2),
 ('eda969b5-a3ef-44d6-97ad-14e540150d7f', 594768, 10, 11356),
 ('10cb651f-7eb7-4b36-bc30-1a2b5a19498c', 1088, 7, 186),
 ('bcc214d3-3b59-4c76-a3d9-6825534ba1b9', 48, 4, 22),
 ('bd520280-a307-4fa0-9145-c9d13f9d01d7', 11347068, 10, 153740)]

In [8]:
labels = ['id', 'reach', 'h', 'n']

In [9]:
reach_df = pd.DataFrame.from_records(reaches, columns=labels)

In [10]:
reach_df.head()

,id,reach,h,n
0,604de395-762d-4a73-b1f2-43a1c417834f,0,1,2
1,332594be-9822-413c-aa98-004fda59e0bd,0,1,2
2,6d27af8a-f73b-4c24-bb9e-0dd3f0d6e66d,7,3,4
3,f478b5dd-1632-47ad-b57a-bee020bc5777,4,1,2
4,575c4a39-d788-43ba-9b59-7e58557cde57,0,1,2


In [11]:
reach_df.sort_values(by='reach', inplace=True, ascending=False)

In [12]:
reach_df.head()

,id,reach,h,n
1188529,a662a4e7-415e-417e-8a8f-fe085d7e487f,35576189665344,10,246505444751
1490908,b49c1e2b-0cd0-4950-a724-00c698e5b49d,23699714081437,10,172037841940
1877125,5ebbd1f5-dfe5-4eec-9883-b8b5efea366c,23366901548285,10,162609805795
1549638,d5e5a24d-f80e-4f1a-b48b-22403b653276,19819022269650,10,138200632114
1633683,25b46064-0d14-4675-a8e7-ab20f1199e54,19016506729398,10,128374417771


In [13]:
reach_df.to_pickle('final_pickles/reach_df_10.pickle')

In [15]:
reach_df[reach_df['id'] == '87d1046b-cbea-4e4e-973b-12a6f6d20b77']

,id,reach,h,n
457928,87d1046b-cbea-4e4e-973b-12a6f6d20b77,3453,8,425
